In [114]:
import sys
import os
from pathlib import Path
from openfermion.hamiltonians import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermionpsi4 import run_psi4

add functions to automate the generation of hamiltonians with varying
number of active spaces, choice of mapping, choice of basis representation.

May need to install the Openfermion-Psi4 package to run:

https://github.com/quantumlib/OpenFermion-Psi4

In [115]:
def generate_and_save(geometry, basis, multiplicity, description, mfilename):
    # initialize the molecule
    molecule = MolecularData(geometry,basis,multiplicity,description=description,filename=mfilename)
    
    # compute the active space integrals
    print('-computing integrals-')
    molecule = run_psi4(molecule,run_mp2=True,run_cisd=True,run_ccsd=True,run_fci=True)
    print(molecule.filename)
    print(molecule.two_body_integrals)
    print(molecule.canonical_orbitals)
    #molecule.save()    
    print('Successful generation')

In [116]:
def load_and_transform(filename, orbitals, transform):
    # Load data
    print('--- loading molecule ---')
    molecule = MolecularData(filename=filename)
    print(molecule.filename)
    #molecule.load()
    
    molecule = run_psi4(molecule,run_mp2=True,run_cisd=True,run_ccsd=True,run_fci=True)

    print('filename: {}'.format(molecule.filename))
    print('n_atoms: {}'.format(molecule.n_atoms))
    print('n_electrons: {}'.format(molecule.n_electrons))
    print('n_orbitals: {}'.format(molecule.n_orbitals))
    print('Canonical Orbitals: {}'.format(molecule.canonical_orbitals))
    print('n_qubits: {}'.format(molecule.n_qubits))
    
    # get the Hamiltonian for a specific choice of active space
    # set the Hamiltonian parameters
    occupied_orbitals, active_orbitals = orbitals
    
    molecular_hamiltonian = molecule.get_molecular_hamiltonian(
                 occupied_indices=range(occupied_orbitals),
                 active_indices=range(active_orbitals))
    
    # map the operator to fermions and then qubits
    fermion_hamiltonian = get_fermion_operator(molecular_hamiltonian)
    
    if transform is 'JW':
        qubit_h = jordan_wigner(fermion_hamiltonian)
        qubit_h.compress()
    elif transform is 'BK':
        qubit_h = bravyi_kitaev(fermion_hamiltonian)
        qubit_h.compress()
    else:
        print('ERROR: Unrecognized qubit transformation: {}'.format(transform))
        sys.exit(2)
        
    return qubit_h

In [117]:
def write_to_file(filename, name, Ne, hamiltonian, description):
    # write the resulting qubit H to file
    print('\n\n~~ writing Qubit Hamiltonian to file~~\n')
    print('filename: {}'.format(filename))
    with open(filename, 'w') as H_file:
        H_file.write('{} {}\n'.format(name, Ne))
        hstring = '{}'.format(hamiltonian)
        terms = hstring.split('\n')
        for t in terms:
            t2 = t.split('[')
            if len(t2) is 2:
                coef = t2[0]
                paul = t2[1].split(']')[0]
                # Check for identity operator
                if paul is '':
                    paul = 'I0'
                    
                # Write coefficients and operators to file
                H_file.write('{0:17s} {1}\n'.format(coef,paul))
                
            else:
                print('ERROR: Something went wrong parsing string')
    print('Successful write\n')

In [118]:
# Main
# Set molecule parameters
name = 'H2'
basis = '6-31g'
multiplicity = 1
num_electrons = 2
transform = 'JW'
num_orbitals = 4

In [119]:
# Can add a for loop here to generate different lengths
# for now, just focus on H2 bond length
bond_length = 0.7414
description = str(round(bond_length,2))
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

if multiplicity is 1:
    mult = 'singlet'
elif multiplicity is 3:
    mult = 'triplet'

molecule_file = 'molecule_data/{}_{}_{}_{}'.format(name,basis,mult,bond_length)

if not os.path.isdir('molecule_data/'):
    os.makedirs('molecule_data/')

print('--- Generate Molecule: {}_{}_{:.2f} ---'.format(name,basis,bond_length))
generate_and_save(geometry, basis, multiplicity, description, molecule_file)

for occupied_num in range(num_orbitals+1):
    for active_num in range(1,num_orbitals+1):
                
        # Load the molecule and perform qubit transformation\
        orbitals = (occupied_num, active_num)
        qubit_h = load_and_transform(molecule_file, orbitals, transform)
        
        # Write the qubit hamiltonian to file
        folder = 'Hamiltonians/'
        if not os.path.isdir(folder):
            os.makedirs(folder)
        fn = '{}_{}_{}_OS{}_AS{}_{}.txt'.format(name,basis,transform,
                                               description,occupied_num,active_num)
        write_to_file(folder+fn,name,num_electrons,qubit_h,description)

--- Generate Molecule: H2_6-31g_0.74 ---
-computing integrals-
molecule_data/H2_6-31g_singlet_0.7414
None
None
Successful generation
--- loading molecule ---


/Users/teague/anaconda3/lib/python3.7/site-packages/openfermionpsi4/_run_psi4.py:226: Warning: No calculation saved. Psi4 segmentation fault possible.
  Warning)


OSError: Unable to open file (unable to open file: name = 'molecule_data/H2_6-31g_singlet_0.7414.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)